In [18]:
import pandas as pd
import numpy as np
from scipy import stats as stats
from scipy import interpolate
import sklearn
from matplotlib import pyplot as plt
import regex
import statsmodels.api as sm
import pickle

In [2]:
df=pd.read_pickle('Final_Train.pkl')
with open('variable_org','rb') as f:
    variable_org=pickle.load(f)
    pass

target=variable_org['target']
dense_factors=variable_org['dense']
sparse_factors=variable_org['sparse']
category_factors=variable_org['categorical']
ordinal_factors=variable_org['ordinal']
other_factors=variable_org['other']

In [9]:
df[target+dense_factors].corr()['price']

price            1.000000
sqft_living      0.694669
sqft_lot         0.092164
sqft_above       0.598673
sqft_basement    0.310118
sqft_living15    0.600390
sqft_lot15       0.082456
age             -0.056035
Name: price, dtype: float64

How about the correlation matrix of all our dense factors.

In [8]:
df[dense_factors].corr()

sqft_living  sqft_lot  sqft_above  sqft_basement  \
sqft_living       1.000000  0.196750    0.873472       0.424741   
sqft_lot          0.196750  1.000000    0.207019       0.018184   
sqft_above        0.873472  0.207019    1.000000      -0.069776   
sqft_basement     0.424741  0.018184   -0.069776       1.000000   
sqft_living15     0.760332  0.184305    0.733713       0.193557   
sqft_lot15        0.187057  0.801512    0.198329       0.014483   
age              -0.324208 -0.070804   -0.428306       0.132137   

               sqft_living15  sqft_lot15       age  
sqft_living         0.760332    0.187057 -0.324208  
sqft_lot            0.184305    0.801512 -0.070804  
sqft_above          0.733713    0.198329 -0.428306  
sqft_basement       0.193557    0.014483  0.132137  
sqft_living15       1.000000    0.210569 -0.333483  
sqft_lot15          0.210569    1.000000 -0.083607  
age                -0.333483   -0.083607  1.000000

In [20]:
fig,ax=plt.subplots(figsize=(10,10))
im=ax.imshow(_8,cmap='Reds')
ax.set_xticklabels(dense_factors)

<ipython-input-20-676b70277e32>:3: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(dense_factors)


[Text(-1.0, 0, 'sqft_living'),
 Text(0.0, 0, 'sqft_lot'),
 Text(1.0, 0, 'sqft_above'),
 Text(2.0, 0, 'sqft_basement'),
 Text(3.0, 0, 'sqft_living15'),
 Text(4.0, 0, 'sqft_lot15'),
 Text(5.0, 0, 'age'),
 Text(6.0, 0, ''),
 Text(7.0, 0, '')]

Let's compare sale prices for the four seasons.

In [3]:
winter=df[df.season=='Winter'].price
fall=df[df.season=='Fall'].price
summer=df[df.season=='Summer'].price
spring=df[df.season=='Spring'].price

In [6]:
df.groupby(['season']).price.agg(['mean','std','count']).round(2)

mean     std  count
season                       
Fall    527.34  348.21   4023
Spring  549.38  357.66   5212
Summer  544.39  346.94   5058
Winter  517.29  348.44   2946

In [7]:
stats.f_oneway(winter,fall,summer,spring)

F_onewayResult(statistic=7.026456253774374, pvalue=0.0001019516682520631)

At a significance level of 0.01, there is evidence to reject.

Next let's look at renovation status

In [26]:
df.groupby(['renovation_status']).price.agg(['mean','std','count']).round(2)

mean     std  count
renovation_status                       
10-24              853.43  552.74    258
2-4                743.75  441.05     34
25+                570.89  399.47    218
5-9                831.28  581.43    105
Never              529.03  340.92  16518
Recent             628.86  310.73    106

In [34]:
frames=()
for status in df.renovation_status.unique():
    if(status!='Never'):
        frames+=(df[df.renovation_status==status].price,)
    else:
        pass
print(len(frames))

5


In [35]:
stats.f_oneway(*frames)

F_onewayResult(statistic=12.604065098274221, pvalue=6.464654778543822e-10)

In [41]:
never=df[df.renovation_status=='Never'].price
renovated=df[df.renovation_status!='Never'].price

In [49]:
two_way_comp=pd.DataFrame([renovated.agg(['mean','std','count']),never.agg(['mean','std','count'])])
two_way_comp.index=['renovated','never']
two_way_comp

mean         std    count
renovated  726.589010  495.072489    721.0
never      529.025546  340.918121  16518.0

In [50]:
stats.ttest_ind(never,renovated)

Ttest_indResult(statistic=-14.890713596540722, pvalue=7.729105167012956e-50)